In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

C:\Users\matze\Documents\Winton\CRISPDM


In [2]:
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test_2.csv').set_index('Id')

### break down 

In [3]:
X_train = df_train.loc[:,'Feature_1':'Ret_120']
y_train = df_train.loc[:,'Ret_121':'Ret_PlusTwo']
weights = df_train[['Weight_Daily','Weight_Intraday']]
X_train_Feature = X_train.loc[:,'Feature_1':'Feature_25']
X_train_Daily = X_train.loc[:,'Ret_MinusTwo':'Ret_MinusOne']
X_train_Minute = X_train.loc[:,'Ret_2':'Ret_120']

In [4]:
X_test = df_test.loc[:,'Feature_1':'Ret_120']

X_test_Feature = X_test.loc[:,'Feature_1':'Feature_25']
X_test_Daily = X_test.loc[:,'Ret_MinusTwo':'Ret_MinusOne']
X_test_Minute = X_test.loc[:,'Ret_2':'Ret_120']

# Typecasting

In [5]:
#X_Feature.head()

In [ ]:
df_train.Feature_1.unique().size
for col in df_train.loc[:, 'Feature_1':'Feature_25'].columns:
    print(col, df_train[col].unique().size, df_test[col].unique().size)

In [6]:

def typecasting(df):
    categorial_cols = ['Feature_1', 'Feature_5', 'Feature_10', 'Feature_20']
    binary_cols = ['Feature_16']

    for col in categorial_cols:
        df[col] = df[col].astype('category')
    
    for col in binary_cols:
        df[col] = (df[col]-1).astype('category')



In [7]:
typecasting(X_train_Feature)
typecasting(X_test_Feature)
#df_train.dtypes

### One Hot encode

In [9]:
X_train_Feature = pd.get_dummies(X_train_Feature)
X_test_Feature = pd.get_dummies(X_test_Feature)

#adding Feature_5 for FEature enineering reasons
X_train_Feature['Feature_5'] = X_train.Feature_5
X_test_Feature['Feature_5'] = X_test.Feature_5

# check for missing values

In [11]:
print('Missing Values in X:: {}'.format(X_train.isna().sum().any()))
print('Missing Values in Target:: {}'.format(y_train.isna().sum().any()))

Missing Values in X:: True
Missing Values in Target:: False


In [13]:
print('Missing Values in X Features:: {}'.format(X_train_Feature.isna().sum().any()))
print('Missing Values in X Daily:: {}'.format(X_train_Daily.isna().sum().any()))
print('Missing Values in X Minute:: {}'.format(X_train_Minute.isna().sum().any()))

Missing Values in X Features:: True
Missing Values in X Daily:: False
Missing Values in X Minute:: True


## missing values in minute returns
Entweder Mittelwert oder 0er

In [ ]:
labels = []
values = []
for col in X_Minute.index:
    labels.append(col)
    values.append(X_Minute.loc[col].isnull().sum()/(X_Minute.columns.size))
print('Percentage of stocks with missing values:: {}'.format((X_Minute.isna().sum(axis=1)!=0).sum()/40000))
pd.DataFrame(values, columns=['Description of Missing Values']).describe()
#der anteil von fehlenden wertenschwankt für die 

In [ ]:
print('Anteil der Werte mit mehr als 10% Fehlenden return rates:: {}'.format(((X_train_Minute.isna().sum(axis=1)/119) > 0.1).sum()/40000))
print('Anteil der Werte mit mehr als 10% Fehlenden return rates:: {}'.format(((X_test_Minute.isna().sum(axis=1)/119) > 0.1).sum()/40000))


### conclusion
Für die Minutenrenditen sind bei den allermeisten werten mehr als 97.5% der WErte vorhanden für diese würde ich einfach mean imputation wählen für die werte die mehr als 10% fehlende werte enthalten muss ggf ein anderen vorgehen gewählt werden oder man schmeißt sie einfach raus

## Missing Values in Features

In [ ]:
labels = []
values = []
for col in X_Feature.columns:
    labels.append(col)
    values.append(X_Feature[col].isnull().sum()/(X_Feature.index.size))
#    print(col, values[-1])
    
train_nan = pd.DataFrame(labels, columns=['labels'])
train_nan['Percentage'] = values
train_nan.loc[train_nan['Percentage'] > 0.1]

### conclusion
some of the columns have lots of missing values i don't think that mean/modus imputation will help a lot there, one option could be to fill in 0 and onehotencode the categorial values

for simplicity reasons i will for now simply fillna(0) the featureset

# Imputation

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

In [15]:
minute_pipeline = Pipeline([
 ('imputer', Imputer(axis=1))
])
feature_pipeline = Pipeline([
 ('imputer', Imputer(axis=0))
])


In [16]:
#train
def imputation(df_feature, df_minute, df_daily):
    df_feature = pd.DataFrame(feature_pipeline.fit_transform(df_feature), columns=df_feature.columns, index = df_feature.index)
    df_minute = pd.DataFrame(minute_pipeline.fit_transform(df_minute),columns=df_minute.columns, index=df_minute.index)
    return pd.concat([df_feature, df_daily, df_minute], axis=1)

In [17]:
X_train_cleaned = imputation(X_train_Feature, X_train_Minute, X_train_Daily)
X_test_cleaned = imputation(X_test_Feature, X_test_Minute, X_test_Daily)

### checking shape and nans

In [18]:
print('Train:: Columns with missing Values:: ',X_train_cleaned.isna().any().sum(), 'Number of Columns:: ', X_train_cleaned.shape[1])
print('Train Target:: Columns with missing Values:: ',y_train.isna().any().sum(), 'Number of Columns:: ', y_train.shape[1])
print('Test:: Columns with missing Values:: ',X_test_cleaned.isna().any().sum(), 'Number of Columns:: ', X_test_cleaned.shape[1])

Train:: Columns with missing Values::  0 Number of Columns::  179
Train Target:: Columns with missing Values::  0 Number of Columns::  62
Test:: Columns with missing Values::  0 Number of Columns::  179


In [26]:
#check if feature 5 is in data for feature engineering
print('Train Feature_5::', X_train_cleaned.columns.isin(['Feature_5']).any())
print('test Feature_5::', X_test_cleaned.columns.isin(['Feature_5']).any())


Train Feature_5:: True
test Feature_5:: True


# Export Cleaned Data

In [27]:
#train
X_train_cleaned.to_csv("X_cleaned.csv")
y_train.to_csv("y_cleaned.csv")

In [28]:
#test
X_test_cleaned.to_csv('Test_cleaned.csv')

In [29]:
#weights
weights.to_csv('weights.csv')